In [1]:
import pandas as pd
import pickle
import numpy as np
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from src.llm_engine.gemini_agent import Agent

#agent = ChatAgent()

In [2]:
df = pd.read_csv(r"../data/GridPuzzle_processed.csv")
df = df[df["id"].isin([13, 821, 687, 718, 350, 1084, 1677, 1437, 1597, 1935, 1020, 1495, 2893, 2208, 2700, 2788, 3720, 3936, 3642, 3659, 4465, 4658, 4087, 4861, 4140, 4464, 4848, 3954, 5740, 5675, 5212, 4976, 5889, 5659, 5325, 5585, 6843, 6292, 6045, 6432, 6823, 5923, 6778, 6997, 7685, 7865, 7183, 7389, 7866, 7188, 7837, 8712, 7959, 8356, 8142, 7993, 8519, 9106, 9719, 9139, 9487, 8853, 8988, 9043, 9497, 9596, 9965, 10037, 10380, 10100, 10450, 10440, 9976, 10251, 11592, 11097, 11022, 11603, 11546, 11748, 11263, 11367, 11407, 11304, 11321, 11923, 12703, 12356, 12535, 12172, 12133, 12395, 12818, 13003, 12900, 13407, 13198, 12782, 13051, 12980])]

In [3]:
df.shape

(100, 15)

In [4]:
from pprint import pprint
pprint(df["Clues"].iloc[0] + "\n"+ df["categories"].iloc[0])

('Clues:\n'
 '1. Stuart lost 4 more pounds than the person who used the vegan diet.\n'
 '2. The dieter who used the gluten-free diet lost 2 more pounds than '
 'Raymond.\n'
 '3. Stuart lost 7 lbs.\n'
 '4. The dieter who lost 7 lbs is either the person who used the caveman diet '
 'or Raymond.\n'
 '5. Wesley lost 5 lbs.\n'
 'pounds lost : 3, 5, 7, 9.\n'
 'names : Raymond, Stuart, Tom, Wesley.\n'
 'diets : caveman, dairy-free, gluten-free, vegan.')


In [ ]:
agent_name = "llm_agent"
# system_prompt = """You are a helpful AI assistant for logic grid puzzles. When given clues and categories:

# Break down each clue into detailed logical steps.
# Look for hidden connections between different clues.
# Explain your reasoning clearly, one step at a time.
# If a clue seems unclear, check your previous steps for helpful information.
# Converge after you have processed all the clues

# Your goal is to help people understand how to solve the puzzle through logical deduction. Show your thought-process only
# without the final answer"""


system_prompt ="""As a logic grid puzzle assistant, analyze relationships by linking category items to relevant clues:

    Item Analysis:

        For each item, identify all applicable clues.

        Note confirmations, eliminations, or constraints.

    Clue Tracking:

        Assign clues to items, flagging conflicts or reinforcements.

    Conclusion Types:

        State definitive connections when clear.

        List possibilities when multiple options exist.

    Review Process:

        Recheck mappings if inconsistencies emerge.

    Completion:

        Stop when all items are fully assessed.

Ensure every item in one category is uniquely mapped to one item from other categories based on categories

You should be constrained based on the given clues.

Goal: Provide conclusive steps about different item mappings by connecting them with clues"""

agent = Agent(agent_name=agent_name,system_prompt=system_prompt)

In [49]:
# print(df["question"].iloc[0])

In [7]:
ind = int(input())

clues = df["Clues"].iloc[ind]
categories = df["categories"].iloc[ind]

user_query = df.question.iloc[0].split("\n")[0] + clues + "\n" + "\n\n"

for index, val in enumerate(categories.split("\n")):
    user_query += f"Category {index +1} \n" + val + "\n"


llm_agent_response = agent.perform_action(user_query)

In [8]:
print(clues)

Clues:
1. Toni was born 9 minutes after Peggy.
2. The baby born at 12:10am was the Blanchards'.
3. Luis wasn't the Vaughans'.
4. Iris was born 3 minutes before the Ortegas' baby.
5. The Ortegas' child was born 9 minutes before the Quinns' baby.
6. Susan was the Blanchards'.


In [9]:
from pprint import pprint
print(llm_agent_response)

Okay, I understand. I made an error in my previous response. I incorrectly assigned Luis to the Quinn family. I will correct this and provide the accurate solution.

**1. Initial Setup:**

We have three categories: Times, Names, and Families. Our goal is to match one item from each category to form a complete set.

**2. Analyzing the Clues:**

*   **Clue 1:** Toni was born 9 minutes after Peggy.
*   **Clue 2:** The baby born at 12:10am was the Blanchards'.
*   **Clue 3:** Luis wasn't the Vaughans'.
*   **Clue 4:** Iris was born 3 minutes before the Ortegas' baby.
*   **Clue 5:** The Ortegas' child was born 9 minutes before the Quinns' baby.
*   **Clue 6:** Susan was the Blanchards'.

**3. Deductions:**

*   **From Clue 6:** Susan = Blanchard
*   **From Clue 2:** Blanchard = 12:10am
*   **Therefore:** Susan = Blanchard = 12:10am

*   **From Clue 4 & 5:** Iris -> Ortega -> Quinn (in that order of birth) with 3 minutes between Iris and Ortega, and 9 minutes between Ortega and Quinn.
*   *

In [21]:
print(df["question"].iloc[0])

A group of friends has decided to try several different weight-loss diets and exercises to see who amongst them can lose the most weight in 3 months. Using only the clues below, match the pounds lost to the options from names and diets. Remember, as with all grid-based logic puzzles, no option in any category will ever be used more than once.

pounds lost : 3, 5, 7, 9.
names : Raymond, Stuart, Tom, Wesley.
diets : caveman, dairy-free, gluten-free, vegan.

Clues:
1. Stuart lost 4 more pounds than the person who used the vegan diet.
2. The dieter who used the gluten-free diet lost 2 more pounds than Raymond.
3. Stuart lost 7 lbs.
4. The dieter who lost 7 lbs is either the person who used the caveman diet or Raymond.
5. Wesley lost 5 lbs.

While answering use the following format:
Step-by-step solution:
Your steps showing how you are solving the puzzle
Final Answer:
Fill the following table to show your final answer.
3 | correct option from names | correct option from diets
5 | correct op

In [17]:
from itertools import combinations

def initialize_probability_tables(num_categories, categories, cat_topics):
    """
    Initialize uniform probability matrices for all pairs of categories.
    
    Parameters:
    num_categories: int - Number of categories
    categories: list - List of category strings with comma-separated subcategories
    
    Returns:
    matrices: dict - Maps category index tuples to probability matrices
    category_mapping: dict - Maps topic strings to category index tuples
    
    Example:
    >>> matrices, mapping = initialize_probability_tables(3, ["red, blue", "small, large", "cheap, expensive"])
    """
    r = 2
    combinations_ls = list(combinations(np.arange(num_categories), r))
    matrices = {}
    for index, combination in enumerate(combinations_ls):
        c1 , c2 = categories[combination[0]].split(", "), categories[combination[1]].split(", ")
       
        matrix =  np.full((len(c1), len(c2)), 1 / max(len(c1), len(c2)))
        topic = cat_topics[combination[0]] + "__x__" + cat_topics[combination[1]]
        matrices[topic] = matrix
       
    return matrices

In [19]:
import re
import json
cat_topics = []
categories = []
cat_str = df["categories"].iloc[0].split("\n")
for item in cat_str:
    cat_topics.append(item.split(" : ")[0])
    categories.append(item.split(" : ")[1])

print(type(cat_topics), cat_topics)

probability_matrices = initialize_probability_tables(len(categories), categories, cat_topics) 


<class 'list'> ['pounds lost', 'names', 'diets']


In [20]:
probability_matrices

{'pounds lost__x__names': array([[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]]),
 'pounds lost__x__diets': array([[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]]),
 'names__x__diets': array([[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]])}

In [10]:
from itertools import combinations
def populate_categories(categories, index):
    
    cat_topics = []
    categories = []

    cat_str = df["categories"].iloc[index].split("\n")
    for item in cat_str:
        cat_topics.append(item.split(" : ")[0])
        categories.append(item.split(" : ")[1].split(", "))

    examples = {}
    r = 2
    combinations_ls = list(combinations(np.arange(len(cat_topics)), r))

    for topic_cob in combinations_ls:
        values = []
        key_1, key_2 = topic_cob
        for i in range(len(categories[0])):
            for j in range(len(categories[0])):

                values.append((categories[key_1][i], categories[key_2][j]))

        examples[cat_topics[topic_cob[0]] + "__x__" + cat_topics[topic_cob[1]]] = values

    return examples

In [11]:
index=ind
examples = populate_categories(categories=categories, index=index)
print(examples)


{'times__x__names': [('12:01am', 'Iris'), ('12:01am', 'Luis'), ('12:01am', 'Peggy'), ('12:01am', 'Susan'), ('12:01am', 'Toni.'), ('12:04am', 'Iris'), ('12:04am', 'Luis'), ('12:04am', 'Peggy'), ('12:04am', 'Susan'), ('12:04am', 'Toni.'), ('12:07am', 'Iris'), ('12:07am', 'Luis'), ('12:07am', 'Peggy'), ('12:07am', 'Susan'), ('12:07am', 'Toni.'), ('12:10am', 'Iris'), ('12:10am', 'Luis'), ('12:10am', 'Peggy'), ('12:10am', 'Susan'), ('12:10am', 'Toni.'), ('12:13am.', 'Iris'), ('12:13am.', 'Luis'), ('12:13am.', 'Peggy'), ('12:13am.', 'Susan'), ('12:13am.', 'Toni.')], 'times__x__families': [('12:01am', 'Blanchard'), ('12:01am', 'Estrada'), ('12:01am', 'Ortega'), ('12:01am', 'Quinn'), ('12:01am', 'Vaughan.'), ('12:04am', 'Blanchard'), ('12:04am', 'Estrada'), ('12:04am', 'Ortega'), ('12:04am', 'Quinn'), ('12:04am', 'Vaughan.'), ('12:07am', 'Blanchard'), ('12:07am', 'Estrada'), ('12:07am', 'Ortega'), ('12:07am', 'Quinn'), ('12:07am', 'Vaughan.'), ('12:10am', 'Blanchard'), ('12:10am', 'Estrada'), 

In [ ]:
# system_prompt = f"""You are a logical reasoning agent.

#                     Given a complex sequential list of logical statements about n-categories and corresponding n-2D matrices depicting inter-category
#                     item mappings, systematically deconstruct each logical deduction. Critically evaluate the relational insights, translating these
#                     abstract logical statements into quantitative probability weights for specific matrix cells. 
#                     The weighting process should be nuanced, reflecting the contextual strength, coherence, and inferential power of each logical
#                     statement as it relates to the underlying categorical relationships.
                    
#                     Comprehending Deductions & Updating Probabilities
                        
#                         Carefully process each logical deduction, ensuring clarity by merging sequential steps when appropriate.
#                         Use conclusive deductions to:

#                             Identify the relevant probability matrix that requires updating.
#                             Determine the exact cells to modify, assigning new weights based on the strength of the deduction.

#                     Dynamic Adjustments

#                         Continuously reassess all decisions as new information becomes available.
#                         If new deductions conflict with prior reasoning, promptly revise the probability matrix or discard incorrect conclusions.
#                         Ensure that all updates reflect the most accurate and up-to-date analysis.

#                     Use these probability values if the information you find fall into one of the categories:
                        
#                         - Definite: 1.0
#                         - Eliminated: 0.0
#                         - Uncertain: 0.5
#                         - Most Likely: 0.6-0.9
                    
#                     Remember, as with all grid-based logic puzzles, no option in any category will ever be used more than once.
#                     Your task is to maintain precision and adaptability, ensuring logical consistency in every update.
#                 """

# agent_job = f"""Process the given  resources below and give the output as expected in the format below.
                
#                 Available Resources:
#                         'Logical Step-By-Step Deductions': {llm_agent_response},
#                         'Initial Probability Matrices': {probability_matrices},
#                         'Category Mapping': {df["categories"].iloc[3]}

#                 - You should use the format below

#                     *** ("matrix_name", probability_value, row_index, column_index)

#                 Return the sequence of probability updates
#             """

In [12]:
system_prompt = f""" You are a probabilistic agent who assess scenarios and logical deductions based on which assign 
                     weightages between 0.05 - 0.95. Your roles include:
                     
                     For every item-item pair given to you, you should assess if they fall under definite, important, 
                     uncertain (any item might have multiple pairs) or useless.
                     
                     Assign a weight for each item-item pair based on the above range.

                     Repeat until all item-item pairs are processed.

                     Return the final answer.

                """
agent_job = f"""Process the given  resources below and give the output as expected in the format below.
                
                Available Resources:
                        'Logical Step-By-Step Deductions': {llm_agent_response},
                        'Item-Item Pair Dictionary': {examples},
                    
            """

In [11]:
categories

'times : 12:01am, 12:04am, 12:07am, 12:10am, 12:13am.\nnames : Iris, Luis, Peggy, Susan, Toni.\nfamilies : Blanchard, Estrada, Ortega, Quinn, Vaughan.'

In [13]:
from pprint import pprint

import json


prob_agent = agent = Agent(agent_name="prob_agent_1", system_prompt=system_prompt, json=True)


prob_agent_resp = prob_agent.perform_action(agent_job)

print(json.loads(prob_agent_resp))



[{'ingredients': [{'Item_Item_Pair': "('12:01am', 'Iris')", 'Probability': 0.95}, {'Item_Item_Pair': "('12:01am', 'Luis')", 'Probability': 0.05}, {'Item_Item_Pair': "('12:01am', 'Peggy')", 'Probability': 0.05}, {'Item_Item_Pair': "('12:01am', 'Susan')", 'Probability': 0.05}, {'Item_Item_Pair': "('12:01am', 'Toni.')", 'Probability': 0.05}, {'Item_Item_Pair': "('12:04am', 'Iris')", 'Probability': 0.05}, {'Item_Item_Pair': "('12:04am', 'Luis')", 'Probability': 0.05}, {'Item_Item_Pair': "('12:04am', 'Peggy')", 'Probability': 0.95}, {'Item_Item_Pair': "('12:04am', 'Susan')", 'Probability': 0.05}, {'Item_Item_Pair': "('12:04am', 'Toni.')", 'Probability': 0.05}, {'Item_Item_Pair': "('12:07am', 'Iris')", 'Probability': 0.05}, {'Item_Item_Pair': "('12:07am', 'Luis')", 'Probability': 0.95}, {'Item_Item_Pair': "('12:07am', 'Peggy')", 'Probability': 0.05}, {'Item_Item_Pair': "('12:07am', 'Susan')", 'Probability': 0.05}, {'Item_Item_Pair': "('12:07am', 'Toni.')", 'Probability': 0.05}, {'Item_Item_P

In [14]:

system_prompt = f"""You are an expert in probability theory and you are given a bunch of item-item pairs and their corresponding
                    probabilities. Conclude the final values of category mappings by forming a grid. It should be in the below format 

                    Final Answer:\n'
                    'Fill your answers as a table.\n'
                                    
                """

agent_job = f"""Analyze the probability matrix and look for the final solution
                The responses are: {prob_agent_resp},
                The categories are: {categories}
"""


In [15]:
solution_agent_system_prompt = system_prompt

solution_agent = Agent(agent_name="solution_agent", system_prompt=solution_agent_system_prompt)


solution_agent_resp = solution_agent.perform_action(agent_job)

print(solution_agent_resp)

```
 Final Answer:
 

 '
 | Time    | Name   | Family    |
 | :------ | :----- | :-------- |
 | 12:01am | Iris   | Ortega    |
 | 12:04am | Peggy  | Estrada   |
 | 12:07am | Luis   | Quinn     |
 | 12:10am | Susan  | Blanchard |
 | 12:13am | Toni.  | Vaughan.  |
 '
 ```


In [16]:
pprint(df["answer"].iloc[ind])

('12:01am | Iris | Vaughan\n'
 '12:04am | Peggy | Ortega\n'
 '12:07am | Luis | Estrada\n'
 '12:10am | Susan | Blanchard\n'
 '12:13am | Toni | Quinn')


In [1]:
from z3 import *


In [2]:

# Variables
Raymond, Stuart, Tom, Wesley = Ints('Raymond Stuart Tom Wesley')
diet_R, diet_S, diet_T, diet_W = Ints('diet_R diet_S diet_T diet_W')

# Diet encodings
caveman, dairy_free, gluten_free, vegan = 0, 1, 2, 3

s = Solver()

# Possible pounds and diets
s.add(Or(Raymond == 3, Raymond == 5, Raymond == 7, Raymond == 9))
s.add(Or(Stuart == 3, Stuart == 5, Stuart == 7, Stuart == 9))
s.add(Or(Tom == 3, Tom == 5, Tom == 7, Tom == 9))
s.add(Or(Wesley == 3, Wesley == 5, Wesley == 7, Wesley == 9))

s.add(Distinct(Raymond, Stuart, Tom, Wesley))
s.add(Distinct(diet_R, diet_S, diet_T, diet_W))

# Clue 3: Stuart lost 7 lbs
s.add(Stuart == 7)

# Clue 5: Wesley lost 5 lbs
s.add(Wesley == 5)

# Clue 1: Stuart lost 4 more than vegan dieter
s.add(Stuart == 4 + If(diet_R == vegan, Raymond,
                     If(diet_S == vegan, Stuart,
                     If(diet_T == vegan, Tom, Wesley))))

# Clue 2: Gluten-free dieter lost 2 more than Raymond
s.add(If(diet_R == gluten_free, Raymond == Raymond + 2,
       If(diet_S == gluten_free, Stuart == Raymond + 2,
       If(diet_T == gluten_free, Tom == Raymond + 2,
          Wesley == Raymond + 2))))

# Clue 4: 7-lb loser is caveman or Raymond
s.add(Or(diet_S == caveman, Raymond == 7))

# Solve
if s.check() == sat:
    m = s.model()
    print(f"Raymond: {m[Raymond]} lbs, diet={m[diet_R]}")
    print(f"Stuart: {m[Stuart]} lbs, diet={m[diet_S]}")
    print(f"Tom: {m[Tom]} lbs, diet={m[diet_T]}")
    print(f"Wesley: {m[Wesley]} lbs, diet={m[diet_W]}")

Raymond: 3 lbs, diet=3
Stuart: 7 lbs, diet=0
Tom: 9 lbs, diet=1
Wesley: 5 lbs, diet=-1


In [5]:
import pandas as pd
df = pd.read_csv(r"../data/GridPuzzle_processed.csv")

In [14]:
from pprint import pprint
res = df[df["id"] == 13]["answer"]
pprint(res)

6    3 | Raymond | vegan\n5 | Wesley | gluten-free\...
Name: answer, dtype: object
